## Truncating Tables

In [26]:
#!/bin/bash

svuota_tabelle(){ 
  lista_tabelle=( temp_ba current_ba )  
  for tabella in "${lista_tabelle[@]}"
  do
     psql -U $PROD_USER -d $PROD_DB -c "TRUNCATE $tabella" 
  done
}

svuota_tabella(){   
  psql -U $PROD_USER -d $PROD_DB -c "TRUNCATE $1"   
}

In [29]:
#svuota_tabelle

#svuota_tabella temp_ba
#svuota_tabella current_ba
svuota_tabella evolution_ba

TRUNCATE TABLE


## Copying Shapefiles 

In [34]:
day=$1

ftp_dir="data/"
filetype="*.shp"

PSQL=/usr/bin/psql
PROD_DB=test_egeos
PROD_USER=postgres

for nomefile in $ftp_dir$filetype
do    
  #echo $nomefile
  fname=$(basename $nomefile)
  #echo $fname
  fbname=${fname%.*}
  echo $fbname    
  #shp2pgsql -a -s 4326 -g geom $nomefile temp_ba | psql -U postgres -d test_egeos
done

20181201_MODA_BA_WGS84
20181202_MODA_BA_WGS84
20181203_MODA_BA_WGS84


In [31]:
CURRENT_ID=$($PSQL -X -U $PROD_USER -h localhost -P t -P format=unaligned $PROD_DB -c "select max(id) from current_ba")
let NEXT_ID=CURRENT_ID+1
echo "Prossimo id per burnt areas current_ba is $NEXT_ID"

Prossimo id per burnt areas current_ba is 1


In [32]:
set -e
set -u

PSQL=/usr/bin/psql

DB_USER=postgres
DB_HOST=localhost
DB_NAME=test_egeos


$PSQL \
      -X \
      -U $DB_USER \
      -h $DB_HOST \
      -c "select id, firedate, area_ha from current_ba" \
      --single-transaction \
      --set AUTOCOMMIT=off \
      --set ON_ERROR_STOP=on \
      --no-align \
      -t \
      --field-separator ' ' \
      --quiet \
      -d $DB_NAME \ | while read -a Record; do
      
      id=${Record[0]}
      firedate=${Record[1]}
      area_ha=${Record[2]}
      
      echo "id: $id, firedate: $firedate, area_hectares: $area_ha"
done